# NYC Collisions and Zoning Districts (Left Join  @ Nearest Zone)
Joining Collision Dataset and Zoning


In [1]:
# Our data management libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import shapefile as shp
from shapely.geometry import Point, Polygon



In [2]:
# Change file paths to match yours to utilize this notebook
zones = gpd.read_file("zoning/zoning.json")
df = pd.read_csv("Story/data/Motor_Vehicle_Collisions_-_Crashes_20231202.csv")

df.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df.reset_index(drop = True)

df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])

C:\Users\Phoenix\AppData\Local\Temp\ipykernel_20688\2276703242.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Story/data/Motor_Vehicle_Collisions_-_Crashes_20231202.csv")


In [3]:
gdf_coordinates = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['LONGITUDE'],df['LATITUDE']),crs='EPSG:4326')
gdf_coordinates.drop(columns = ['LOCATION'], inplace = True)

In [4]:
print(f'CRS of zones: {zones.crs}')
print(f'CRS of collisions: {gdf_coordinates.crs}')

CRS of zones: EPSG:4326
CRS of collisions: EPSG:4326


In [5]:
result = gpd.sjoin_nearest(gdf_coordinates, zones, how="left")

c:\Users\Phoenix\miniconda3\envs\fws-datasci\Lib\site-packages\geopandas\array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [6]:
result.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,geometry,index_right,OBJECTID,ZONEDIST,Shape__Area,Shape__Length
3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,POINT (-73.86650 40.66720),164,165,R6,1.553141e+06,5916.583305
4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,POINT (-73.91727 40.68330),251,252,PARK,1.424097e+05,1804.383968
6,2021-12-14,17:05,NaN,NaN,40.709183,-73.956825,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0.0,...,Tractor Truck Diesel,NaN,NaN,NaN,POINT (-73.95682 40.70918),197,198,PARK,8.611491e+03,506.562422
7,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.831480,NaN,NaN,344 BAYCHESTER AVENUE,2.0,...,Sedan,NaN,NaN,NaN,POINT (-73.83148 40.86816),2318,2319,M1-1,2.733373e+06,7499.349240
8,2021-12-14,21:10,BROOKLYN,11207.0,40.671720,-73.897100,NaN,NaN,2047 PITKIN AVENUE,0.0,...,NaN,NaN,NaN,NaN,POINT (-73.89710 40.67172),1749,1750,M3-2,8.868973e+05,4538.967156


In [7]:
def replaceZoneDistrict(entry):
    if "R" and "M" in entry:
        return "Mixed Manufacturing/Residential"
    elif "PARK" == entry:
        return "Park"
    elif "BPC" == entry:
        return "Battery Park City"
    elif "C" in entry:
        return "Commercial"
    elif "M" in entry:
        return "Manufacturing"
    elif "R" in entry:
        return "Residential"

In [8]:
result['ZONEDIST'] = result['ZONEDIST'].apply(replaceZoneDistrict)

In [9]:
result.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5', 'geometry', 'index_right', 'OBJECTID',
       'ZONEDIST', 'Shape__Area', 'Shape__Length'],
      dtype='object')

In [10]:
columnsToDrop = ['index_right','OBJECTID','Shape__Area','Shape__Length','geometry']

In [11]:
result.drop(columns = columnsToDrop, inplace = True)

In [12]:
result['ZONEDIST'].value_counts()

ZONEDIST
Residential                        827828
Commercial                         391599
Park                               367425
Mixed Manufacturing/Residential    227375
Battery Park City                     571
Name: count, dtype: int64

In [13]:
result.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,ZONEDIST
3,2021-09-11,9:35,BROOKLYN,11208.0,40.667202,-73.866500,NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,Residential
4,2021-12-14,8:13,BROOKLYN,11233.0,40.683304,-73.917274,SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,Park
6,2021-12-14,17:05,NaN,NaN,40.709183,-73.956825,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0.0,...,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN,Park
7,2021-12-14,8:17,BRONX,10475.0,40.868160,-73.831480,NaN,NaN,344 BAYCHESTER AVENUE,2.0,...,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN,Mixed Manufacturing/Residential
8,2021-12-14,21:10,BROOKLYN,11207.0,40.671720,-73.897100,NaN,NaN,2047 PITKIN AVENUE,0.0,...,NaN,NaN,NaN,4487074,Sedan,NaN,NaN,NaN,NaN,Mixed Manufacturing/Residential


In [14]:
result.to_csv('data/collisions_and_zoningDistricts.csv')